In [1]:
import os
import pandas as pd
import numpy as np
import re

from keras.preprocessing.text import text_to_word_sequence

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.utils import to_categorical


Using TensorFlow backend.


In [12]:
from sklearn.utils import shuffle

In [3]:
data_csv = pd.read_csv('/home/abhinav/data/full_text/alan_cleaner/filter_info.csv')

In [65]:
np_test = data_csv[(data_csv.label == 0) & (data_csv.is_protest == False)]
np_test_tags = np_test.filename.values
np.random.seed(0)
np_test_tags = shuffle(np_test_tags)[:45000]

In [73]:
np_train_filter = data_csv[(data_csv.label == 0)&(data_csv.is_protest == True)] 
np_train_tags = np_train_filter.filename.values

In [75]:
p_train = data_csv[data_csv.label==1]

In [76]:
np.random.seed(0)
p_train = shuffle(p_train)

In [77]:
p_test = p_train[:500]
p_train = p_train[500:]

In [78]:
p_train_tags = p_train.filename.values
p_test_tags = p_test.filename.values

In [52]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [84]:
TEXT_DATA_DIR = "/home/abhinav/data/full_text"
text_neg_train = []
text_neg_test = []
count = 0
for name in sorted(os.listdir(TEXT_DATA_DIR+"/not_protest")):
    path = os.path.join(TEXT_DATA_DIR+'/not_protest', name)
    f = open(path)
    t = f.read()
    t = t.split("</DOC>")[:-1]
    for i,docs in enumerate(t):
        id_ = re.search(r"id=\"(.*)\" type",t[i]).group(1)[-7:]+".txt"
        if (count %10000) == 0:
            print (count," Done")
        if id_ in np_train_tags:
            t[i] = re.sub(r"((\<.*\>)|(\d+)|(LEAD:(.*)\n))","",t[i])
            t[i] = [j for j in t[i].lower().split() if j not in stop]
            t[i] = " ".join(t[i])
            t[i] = " ".join(text_to_word_sequence(t[i]))
            text_neg_train.append(t[i])
        elif id_ in np_test_tags:
            t[i] = re.sub(r"((\<.*\>)|(\d+)|(LEAD:(.*)\n))","",t[i])
            t[i] = [j for j in t[i].lower().split() if j not in stop]
            t[i] = " ".join(t[i])
            t[i] = " ".join(text_to_word_sequence(t[i]))
            text_neg_test.append(t[i])
        count = count +1

(0, ' Done')
(10000, ' Done')
(20000, ' Done')
(30000, ' Done')
(40000, ' Done')
(50000, ' Done')
(60000, ' Done')
(70000, ' Done')
(80000, ' Done')
(90000, ' Done')
(100000, ' Done')
(110000, ' Done')
(120000, ' Done')
(130000, ' Done')
(140000, ' Done')
(150000, ' Done')


In [85]:
TEXT_DATA_DIR = "/home/abhinav/data/full_text"
text_pos_train = []
text_pos_test = []

for name in sorted(os.listdir(TEXT_DATA_DIR+"/protest")):
    path = os.path.join(TEXT_DATA_DIR+'/protest', name)
    f = open(path)
    t = f.read()
    t = t.split("</DOC>")[:-1]
    for i,docs in enumerate(t):
        id_ = re.search(r"id=\"(.*)\" type",t[i]).group(1)[-7:]+".txt"        
        if id_ in p_train_tags:
            t[i] = re.sub(r"((\<.*\>)|(\d+)|(LEAD:(.*)\n))","",t[i])
            t[i] = [j for j in t[i].lower().split() if j not in stop]
            t[i] = " ".join(t[i])
            t[i] = " ".join(text_to_word_sequence(t[i]))
            text_pos_train.append(t[i])
        elif id_ in p_test_tags:
            t[i] = re.sub(r"((\<.*\>)|(\d+)|(LEAD:(.*)\n))","",t[i])
            t[i] = [j for j in t[i].lower().split() if j not in stop]
            t[i] = " ".join(t[i])
            t[i] = " ".join(text_to_word_sequence(t[i]))
            text_pos_test.append(t[i])
        

In [90]:
SIXTH_MOD_DIR = "/home/abhinav/data/sixth_model_alan_filtering/"
MOD_DIR = SIXTH_MOD_DIR

In [104]:
BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 100

In [92]:
text_data  =  text_neg_train + text_pos_train 

In [93]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)

In [94]:
pd.to_pickle(tokenizer,MOD_DIR+'tokenizer')

In [95]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 103916 unique tokens.


In [97]:
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', x_train.shape)

label_neg_train = np.zeros((len(text_neg_train),),dtype=int)
label_pos_train = np.ones((len(text_pos_train),),dtype=int)
labels_train = np.append(label_neg_train,label_pos_train)
y_train = to_categorical(np.asarray(labels_train))

pd.to_pickle([x_train,y_train],MOD_DIR+'train_data')

In [98]:
test_seq = tokenizer.texts_to_sequences(text_pos_test)
x_test_pos = pad_sequences(test_seq,maxlen = MAX_SEQUENCE_LENGTH)

y_test_positives = np.ones((len(text_pos_test),))
temp = np.append(np.zeros((1,)),y_test_positives)
y_test_positives = to_categorical(temp)[1:]

test_seq_ = tokenizer.texts_to_sequences(text_neg_test)
x_test_neg = pad_sequences(test_seq_,maxlen = MAX_SEQUENCE_LENGTH)

y_test_negatives = np.zeros((len(text_neg_test),))
temp = np.append(np.ones((1,)),y_test_negatives)
y_test_negatives = to_categorical(temp)[1:]

x_test = np.append(x_test_neg,x_test_pos,axis=0)
y_test = to_categorical(np.append(np.zeros((len(text_neg_test),)),np.ones((len(text_pos_test),))))



pd.to_pickle(obj=[x_test,y_test], path=MOD_DIR + "test_dataset")


In [102]:
print('Indexing word vectors.')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

Indexing word vectors.


In [105]:
print('Preparing embedding matrix.')
#embeddings_index = pd.read_pickle('/home/abhinav/data/fourth_model/embedding_hash_300d')
# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


pd.to_pickle(embedding_matrix,MOD_DIR + "embedding_matrix")

Preparing embedding matrix.
